In [36]:
import pandas as pd
import random
from statistics import mean
from transformers import T5Tokenizer
from src.tfr_decoding.shp_modeling import T5BinaryClassifier

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
tokenizer = T5Tokenizer.from_pretrained("google/flan-t5-xxl")

In [38]:
pfmod_path = "./lightning_logs/multimodel/checkpoints/epoch=3-step=86946.ckpt"
pfname = 'stanfordnlp/SteamSHP-flan-t5-large'
# get prefix model
qpref = T5BinaryClassifier.load_from_checkpoint(pfmod_path).to("cuda:1")
preftok = T5Tokenizer.from_pretrained(pfname)

/home/prasann/miniconda3/envs/rewardenv/lib/python3.8/site-packages/pytorch_lightning/utilities/migration/utils.py:49: PossibleUserWarning: The loaded checkpoint was produced with Lightning v2.0.2, which is newer than your current Lightning version: v2.0.1.post0
  rank_zero_warn(


In [4]:
# load in data
d1 = pd.read_json("output/testset1.jsonl", orient="records", lines=True).drop(columns=["stats", 'ver', 'pref'])
d2 = pd.read_json("output/testset2.jsonl", orient="records", lines=True).drop(columns=["stats", 'ver', 'pref'])
fulld = pd.DataFrame({'inp':d1.inp, 'hyps':[d1['hyps'][i]+d2['hyps'][i] for i in range(len(d1))], 'scos':[d1['scos'][i]+d2['scos'][i] for i in range(len(d1))]})

In [29]:
def make_toklen_lists(indf, tok):
    res = []
    for h in indf.hyps:
        lens = [len(tok(hyp).input_ids) for hyp in h]
        res.append(lens)
    return res

def make_prefs(indf, tok, pflen):
    res = []
    for h in indf.hyps:
        prefs = [tok.decode(tok(hyp).input_ids[:pflen], skip_special_tokens=True) for hyp in h]
        res.append(prefs)
    return res

def adaptbase(inpdf, thresh, mrange):
    inds = list(range(8))
    fscos = []
    budgets = []
    for ind in range(len(inpdf)):
        slist = inpdf.scos[ind]
        blist = inpdf.toklens[ind]
        random.shuffle(inds)
        tmp = []
        budget = 0
        # get adaptive score
        for i in inds[:mrange]:
            budget = budget+blist[i]
            if slist[i]>thresh:
                fscos.append(slist[i])
                break
            tmp.append(slist[i])
            if i==inds[mrange-1]:
                fscos.append(max(tmp))
        budgets.append(budget)
    return mean(fscos), mean(budgets)

In [6]:
fulld['toklens'] = make_toklen_lists(fulld, tokenizer)

In [32]:
fulld['pf5'] = make_prefs(fulld, tokenizer, 5)
fulld['pf10'] = make_prefs(fulld, tokenizer, 10)
fulld['pf15'] = make_prefs(fulld, tokenizer, 15)
fulld['pf20'] = make_prefs(fulld, tokenizer, 20)

In [34]:
fulld.to_json("output/adapt_explore.jsonl", orient='records', lines=True)

In [41]:
qpref.predsingle(fulld['inp'][0], fulld['pf10'][0][0], True)

GreedySearchEncoderDecoderOutput(sequences=tensor([[0, 1]], device='cuda:1'), scores=(tensor([[14.7909, 20.2716, -0.4009,  ...,  0.8677,  0.9384,  0.9633]],
       device='cuda:1'),), encoder_attentions=None, encoder_hidden_states=None, decoder_attentions=None, cross_attentions=None, decoder_hidden_states=None)

In [24]:
# run adaptive baseline
for j in range(2, 9):
    scos, buds = [], []
    for i in range(1000):
        s, b = adaptbase(fulld, .8, j)
        scos.append(s)
        buds.append(b)
    print(mean(scos), " ", mean(buds))

0.8215939434854235   106.08260413080895
0.8415627102416797   126.43164888123924
0.8510720460971218   139.83847676419967
0.8564655803450021   149.46631841652322
0.8599671189500544   156.67206884681585
0.8622596561108932   162.50760413080894
0.8640506297692907   167.20613425129088


In [65]:
mean([max(m) for m in fulld.scos])

0.9002139297487092

In [ ]:
0.8155690845482949   97.05854561101549
0.8310641830621897   109.04165748709123
0.8378271147807699   116.05017039586919
0.8412764822139855   120.31467814113597
0.8434293656394016   123.2454526678141
0.8447807333112877   125.28040275387264
0.8457303369521343   126.81544578313253